In [1]:
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import seaborn as sb
import plotly
import dash
import plotly.plotly as ply
from plotly.graph_objs import *
from scipy.io import netcdf
%matplotlib inline

In [2]:
plotly.tools.set_credentials_file(username='jpterry', api_key='jC0hHrhZQhOys3ypgeg8')

In [3]:
origins = ["DR_Congo", "Afghanistan", "Syria", "Myanmar", "Sudan"]
destinations = ["USA", "UK", "France", "Canada", "Italy", "Germany"]

org_codes = ["COD", "AFG", "SYR", "MMR", "SDN"]
dest_codes = ["USA", "GBR", "FRA", "CAN", "ITA","DEU"]

years = [2000 + i for i in range(19)]

features = ['destination',
 'Origin',
 'applied',
 'accepted',
 'Rejected',
 'decisions',
 'Year',
 'Month',
 'Value',
 'deaths',
 'last_month',
 'two_months_ago',
 'next_month',
 'two_months_later']

pairs = []
for origin in origins:
    for destination in destinations:
        pairs.append((origin, destination))

In [4]:
# Read in data
groups = {}
for pair in pairs:
    groups[pair] = pd.read_csv('./data/grouped_' + str(pair) + '.csv', skipinitialspace=True)
    groups[pair] = groups[pair][features]

In [5]:
sent = np.array([])
for origin in origins:
    total = 0
    for destination in destinations:
        total += groups[(origin, destination)].Value.sum()
    sent = np.append(sent, total)
    
avg_sent = np.array([])
for origin in origins:
    total = 0
    for destination in destinations:
        total += groups[(origin, destination)].Value.sum()/(groups[pair].Year.max() - groups[pair].Year.min())
    avg_sent = np.append(avg_sent, total)

In [36]:
dead = np.array([])
for origin in origins:
    total = 0
    for destination in destinations:
        total += sum(groups[(origin, destination)].deaths.unique())
        
    dead = np.append(dead, total)
    
avg_dead = np.array([])
for origin in origins:
    total = 0
    for destination in destinations:
        total += sum(groups[(origin, destination)].deaths.unique())/(groups[pair].Year.max() - groups[pair].Year.min())
    avg_dead = np.append(avg_dead, total)

In [8]:
this_year = 2000.
a = groups[(origin, destination)].query('Year == @this_year').deaths.unique()[0]

In [9]:
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec"]

combos = []
for year in years:
    for month in months:
        combos.append((month, year))

In [37]:
dead_each_year = []
for year in years:
    this_year = []
    for origin in origins:
        total = 0
        for destination in destinations:
            try:
                total += groups[(origin, destination)].query('Year == @year').deaths.unique()[0]
                break
            except:
                total += 0
        this_year.append(total)
    dead_each_year.append(this_year)

In [ ]:
dead_each_year

In [38]:
arrived = np.array([])
for destination in destinations:
    total = 0
    for origin in origins:
        total += groups[(origin, destination)].Value.sum()
    arrived = np.append(arrived, total)

avg_arrived = np.array([])
for destination in destinations:
    total = 0
    for origin in origins:
        total += groups[(origin, destination)].Value.sum()/(groups[pair].Year.max() - groups[pair].Year.min())
    avg_arrived = np.append(avg_arrived, total)

In [14]:
sent, arrived, dead, len(avg_arrived)

(array([ 97774., 337899., 584777.,  11655.,  59188.]),
 array([ 24909., 101881., 131539.,  23039.,  28454., 781471.]),
 array([  950240., 10851299., 26695006.,   276607.,  2443237.]),
 6)

In [16]:
coordinates = {}
coordinates["Syria"] = (34.8, 38.997)
coordinates["DR_Congo"] = (-4.0383, 21.759)
coordinates["Afghanistan"] = (33.939, 67.71)
coordinates["Sudan"] = (12.863, 30.218)
coordinates["Myanmar"] = (21.9162, 95.956)

coordinates["USA"] = (37.09, -95.713)
coordinates["UK"] = (55.378, -3.436)
coordinates["France"] = (46.228, 2.214)
coordinates["Germany"] = (51.166, 10.4515)
coordinates["Italy"] = (41.872, 12.5674)
coordinates["Canada"] = (56.1304, -106.3468)

In [17]:
dest_lats = [coordinates[dest][0] for dest in destinations] 
dest_lons = [coordinates[dest][1] for dest in destinations] 
org_lats = [coordinates[org][0] for org in origins] 
org_lons = [coordinates[org][1] for org in origins]

In [18]:
org_texts =[origins[i] + 'Total deaths = ' + str(dead[i]) for i in range(len(dead))]

In [20]:
sent_each_year_by_orig = []
relative_sizes = []
for origin in origins:
    this_origin = []
    these_sizes = []
    for year in years:
        total = 0
        year = int(year)
        for destination in destinations:
            try:
                total += groups[(origin, destination)].query('Year == @year').Value.sum()
            except:
                total += 0
        this_origin.append(total)
    for i in range(len(years)):
        these_sizes.append(0.5e2*this_origin[i]/max(this_origin))
    sent_each_year_by_orig.append(this_origin)
    relative_sizes.append(these_sizes)

In [ ]:
relative_sizes2 = [[] for i in range(len(years))]
for i in range(len(relative_sizes)):
    for ii in range(len(relative_sizes[i])):
        relative_sizes2[ii].append(relative_sizes[i][ii])

In [21]:
sent_each_year = [[] for i in range(len(years))]
year_sizes = [[] for i in range(len(years))]
for i in range(len(sent_each_year_by_orig)):
    for ii in range(len(sent_each_year_by_orig[i])):
        sent_each_year[ii].append(sent_each_year_by_orig[i][ii])
#         year_sizes[ii].append(relative_sizes[i][ii])
for i in range(len(sent_each_year)):
    for ii in range(len(sent_each_year[i])):
        if max(sent_each_year[i]) != 0:
            year_sizes[i].append(50*sent_each_year[i][ii]/max(sent_each_year[i]))
        else:
            year_sizes[i].append(0)

In [155]:
arr_each_year_by_dest = []
relative_sizes_arr = []
for destination in destinations:
    this_dest = []
    these_sizes = []
    for year in years:
        total = 0
        year = int(year)
        for origin in origins:
            try:
                total += groups[(origin, destination)].query('Year == @year').Value.sum()
            except:
                total += 0
        this_dest.append(total)
    for i in range(len(years)):
        these_sizes.append(0.5e2*this_dest[i]/max(this_dest))
    arr_each_year_by_dest.append(this_dest)
    relative_sizes_arr.append(these_sizes)

In [156]:
relative_sizes_arr2 = [[] for i in range(len(years))]
for i in range(len(relative_sizes_arr)):
    for ii in range(len(relative_sizes_arr[i])):
        relative_sizes_arr2[ii].append(relative_sizes_arr[i][ii])

In [157]:
arr_each_year = [[] for i in range(len(years))]
for i in range(len(arr_each_year_by_dest)):
    for ii in range(len(arr_each_year_by_dest[i])):
        arr_each_year[ii].append(arr_each_year_by_dest[i][ii])
#         year_sizes[ii].append(relative_sizes[i][ii])

In [14]:
sent, arrived, dead, len(avg_arrived)

(array([ 97774., 337899., 584777.,  11655.,  59188.]),
 array([ 24909., 101881., 131539.,  23039.,  28454., 781471.]),
 array([  950240., 10851299., 26695006.,   276607.,  2443237.]),
 6)

In [23]:
dead_dic = {}
for i in range(len(years)-1):
    dead_dic[years[i]] = dead_each_year[i]

In [ ]:
dead_dic

In [24]:
df = pd.DataFrame(dead_dic, index=origins)

In [25]:
year = 2010
df[df.index == 'Syria'][year]

Syria    1.000000e-16
Name: 2010, dtype: float64

In [26]:
sent_dic = {}
for i in range(len(years)-1):
    sent_dic[years[i]] = sent_each_year[i]

In [27]:
df_s = pd.DataFrame(sent_dic, index=origins)

In [28]:
# filtered_df_s = df_s.loc[:, 2010]
df_s.loc["Syria"]

2000      2380.0
2001      2571.0
2002      2030.0
2003      1603.0
2004      1379.0
2005      1436.0
2006       937.0
2007       960.0
2008      1127.0
2009      1285.0
2010      2077.0
2011      4240.0
2012      9513.0
2013     17884.0
2014     47581.0
2015    168761.0
2016    276351.0
2017     42662.0
Name: Syria, dtype: float64

In [29]:
# i = "Syria"
# sent_by_country = filtered_df_s[filtered_df_s.index == i]

NameError: name 'filtered_df_s' is not defined

In [ ]:
sent_by_country + 2

### Start plots

#### Interactive Maps

In [39]:
data = [dict(
        type = 'choropleth',
        locations = org_codes,
        z = dead,
        text = origins,
        colorscale = [[0,"rgb(139,0,0)"],[0.35,"rgb(255,0,0)"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,255,0)"], [1, "rgb(255,255,204)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Conflict Deaths'),
      ) ]

layout = dict(
    title = 'Total Conflict Deaths 2000 - 2018',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
ply.iplot(fig, validate=False, filename='deaths_world' )

In [40]:
data = [dict(
        type = 'choropleth',
        locations = org_codes,
        z = avg_dead,
        text = origins,
        colorscale = [[0,"rgb(139,0,0)"],[0.35,"rgb(255,0,0)"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,255,0)"], [1, "rgb(255,255,204)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Conflict Deaths'),
      ) ]

layout = dict(
    title = 'Conflict Deaths per Year',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
ply.iplot(fig, validate=False, filename='avg_deaths_world' )

In [41]:
data = [dict(
        type = 'choropleth',
        locations = org_codes,
        z = sent,
        text = origins,
        colorscale = [[0,"rgb(255,0,0)"],[0.35,"rgb(255,99,71))"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,215,0"],[1,"rgb(255,255,102)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Total Refugees Deaths'),
      ) ]

layout = dict(
    title = 'Total Refugees Sent 2000 - 2018',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)

data2 = [dict(
        type = 'choropleth',
        locations = dest_codes,
        z = arrived,
        text = destinations,
        colorscale = [[0,"rgb(255,0,0)"],[0.35,"rgb(255,99,71))"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,255,0)"], [1, "rgb(255,255,224)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Refugees'),
      ) ]

layout2 = dict(
    title = 'Total Refugees Accepted 2000 - 2018',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)



fig = dict(data=data, layout=layout)
fig2 = dict(data=data2, layout=layout2)
ply.iplot(fig2, validate=False, filename='total_refugees_accepted' )

In [42]:
ply.iplot(fig, validate=False, filename='total_refugees_sent' )

In [43]:
data = [dict(
        type = 'choropleth',
        locations = org_codes,
        z = avg_sent,
        text = origins,
        colorscale = [[0,"rgb(255,0,0)"],[0.35,"rgb(255,99,71))"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,215,0"],[1,"rgb(255,255,102)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Refugees Sent per Year'),
      ) ]

layout = dict(
    title = 'Refugees Sent per Year',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)

In [44]:
fig = dict(data=data, layout=layout)
ply.iplot(fig, validate=False, filename='avg_refugees_sent' )

In [45]:
data = [dict(
        type = 'choropleth',
        locations = dest_codes,
        z = avg_arrived,
        text = destinations,
        colorscale = [[0,"rgb(255,0,0)"],[0.35,"rgb(255,99,71))"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,215,0"],[1,"rgb(255,255,102)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Refugees Accepted per Year'),
      ) ]

layout = dict(
    title = 'Refugees Accepted per Year',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)

In [46]:
fig = dict(data=data, layout=layout)
ply.iplot(fig, validate=False, filename='avg_refugees_accepted' )

#### Refugees each year

In [151]:
import plotly.graph_objs as go
sent_year = []
# colors = ['rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)']
# colors = 'inferno'

labels = []
for i in range(len(sent_each_year)):
    labels.append([])
    for ii in range(len(origins)):
        labels[i].append(str(sent_each_year[i][ii]) + ' ' + origins[ii])

colors = ['red','crimson', 'deeppink', 'fuchsia', 'violet', \
         'darkviolet', 'slateblue', 'darkblue','slategrey', \
          'skyblue', 'darkslategrey',
         'mediumseagreen', 'green','lawngreen', 'orange', 'peachpuff',\
         'coral', 'lightcoral']

for i in range(len(years)-1):
    sent_year.append(go.Scattergeo(
        lon = org_lons,
        lat = org_lats,
        text = labels[i],
        name = str(years[i]),
        marker = dict(
            size = relative_sizes2[i],
            color = colors[i]
#             line = dict(width = 0)
        ),
    ) )

# cases[0]['text'] = df[ df['Month'] == 9 ]['Value'].map('{:.0f}'.format).astype(str)+' '+\
#     df[ df['Month'] == 9 ]['Country']
# cases[0]['mode'] = 'markers+text'
# cases[0]['textposition'] = 'bottom center'

inset = [
    go.Choropleth(
#         locationmode = 'country codes',
        locations = org_codes,
        z = sent_each_year,
#         text = df[ df['Month'] == 9 ]['Country'],
        colorscale = [[0,'rgb(0, 0, 0)'],[1,'rgb(0, 0, 0)']],
        autocolorscale = False,
        showscale = False,
        geo = 'geo'
    ),
    go.Scattergeo(
        lon = [21.0936],
        lat = [7.1881],
        text = ['World'],
        mode = 'text',
        showlegend = False,
        geo = 'geo'
    )
]

layout = go.Layout(
    title = 'Regufees each year',
    geo = dict(
        resolution = 50,
        scope = 'world',
        showframe = False,
        showcoastlines = True,
        showland = True,
        showcountries=True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'mercator'
        )#,
#         lonaxis = dict( range= [ -15.0, -5.0 ] ),
#         lataxis = dict( range= [ 0.0, 12.0 ] ),
#         domain = dict(
#             x = [ 0, 1 ],
#             y = [ 0, 1 ]
#         )
    )#,
#     geo2 = dict(
#         scope = 'world',
#         showframe = False,
#         showland = True,
#         landcolor = "rgb(229, 229, 229)",
#         showcountries = False,
#         domain = dict(
#             x = [ 0, 0.6 ],
#             y = [ 0, 0.6 ]
#         ),
#         bgcolor = 'rgba(255, 255, 255, 0.0)',
#     ),
#     legend = dict(
#            traceorder = 'reversed'
#     )
)

fig = go.Figure(layout=layout, data=sent_year)
ply.iplot(fig, validate=False, filename='Refugees_sent_each_year')

In [159]:
import plotly.graph_objs as go
arr_year = []
# colors = ['rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)']
# colors = 'inferno'

labels = []
for i in range(len(arr_each_year)):
    labels.append([])
    for ii in range(len(destinations)):
        labels[i].append(str(arr_each_year[i][ii]) + ' ' + destinations[ii])

colors = ['red','crimson', 'deeppink', 'fuchsia', 'violet', \
         'darkviolet', 'slateblue', 'darkblue','slategrey', \
          'skyblue', 'darkslategrey',
         'mediumseagreen', 'green','lawngreen', 'orange', 'peachpuff',\
         'coral', 'lightcoral']

for i in range(len(years)-1):
    arr_year.append(go.Scattergeo(
        lon = dest_lons,
        lat = dest_lats,
        text = labels[i],
        name = str(years[i]),
        marker = dict(
            size = relative_sizes_arr2[i],
            color = colors[i]
#             line = dict(width = 0)
        ),
    ) )

# cases[0]['text'] = df[ df['Month'] == 9 ]['Value'].map('{:.0f}'.format).astype(str)+' '+\
#     df[ df['Month'] == 9 ]['Country']
# cases[0]['mode'] = 'markers+text'
# cases[0]['textposition'] = 'bottom center'

inset = [
    go.Choropleth(
#         locationmode = 'country codes',
        locations = dest_codes,
        z = arr_each_year,
#         text = df[ df['Month'] == 9 ]['Country'],
        colorscale = [[0,'rgb(0, 0, 0)'],[1,'rgb(0, 0, 0)']],
        autocolorscale = False,
        showscale = False,
        geo = 'geo'
    ),
    go.Scattergeo(
        lon = [21.0936],
        lat = [7.1881],
        text = ['World'],
        mode = 'text',
        showlegend = False,
        geo = 'geo'
    )
]

layout = go.Layout(
    title = 'Regufees each year',
    geo = dict(
        resolution = 50,
        scope = 'world',
        showframe = False,
        showcoastlines = True,
        showland = True,
        showcountries=True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'mercator'
        )#,
#         lonaxis = dict( range= [ -15.0, -5.0 ] ),
#         lataxis = dict( range= [ 0.0, 12.0 ] ),
#         domain = dict(
#             x = [ 0, 1 ],
#             y = [ 0, 1 ]
#         )
    )#,
#     geo2 = dict(
#         scope = 'world',
#         showframe = False,
#         showland = True,
#         landcolor = "rgb(229, 229, 229)",
#         showcountries = False,
#         domain = dict(
#             x = [ 0, 0.6 ],
#             y = [ 0, 0.6 ]
#         ),
#         bgcolor = 'rgba(255, 255, 255, 0.0)',
#     ),
#     legend = dict(
#            traceorder = 'reversed'
#     )
)

fig = go.Figure(layout=layout, data=arr_year)
ply.iplot(fig, validate=False, filename='Refugees_taken_each_year')

In [113]:
colors2

[[array([0.96862745, 0.98823529, 0.99215686, 1.        ]),
  array([0.96862745, 0.98823529, 0.99215686, 1.        ]),
  array([0.96862745, 0.98823529, 0.99215686, 1.        ]),
  array([0.96862745, 0.98823529, 0.99215686, 1.        ]),
  array([0.96862745, 0.98823529, 0.99215686, 1.        ])],
 [array([0.92901192, 0.96067666, 0.97665513, 1.        ]),
  array([0.92901192, 0.96067666, 0.97665513, 1.        ]),
  array([0.92901192, 0.96067666, 0.97665513, 1.        ]),
  array([0.92901192, 0.96067666, 0.97665513, 1.        ]),
  array([0.92901192, 0.96067666, 0.97665513, 1.        ])],
 [array([0.88939639, 0.93311803, 0.9611534 , 1.        ]),
  array([0.88939639, 0.93311803, 0.9611534 , 1.        ]),
  array([0.88939639, 0.93311803, 0.9611534 , 1.        ]),
  array([0.88939639, 0.93311803, 0.9611534 , 1.        ]),
  array([0.88939639, 0.93311803, 0.9611534 , 1.        ])],
 [array([0.83732411, 0.89434833, 0.9394233 , 1.        ]),
  array([0.83732411, 0.89434833, 0.9394233 , 1.      

In [81]:
sent_each_year_by_orig[1]

[11658.0,
 15930.0,
 11308.0,
 4582.0,
 2357.0,
 2504.0,
 3557.0,
 3699.0,
 6810.0,
 8756.0,
 9931.0,
 11992.0,
 11315.0,
 12338.0,
 15315.0,
 41486.0,
 139350.0,
 25011.0,
 0.0]

In [88]:
labels2 = []
for i in range(len(sent_each_year_by_orig)):
    labels2.append([])
    for ii in range(len(years)):
        labels2[i].append(str(sent_each_year_by_orig[i][ii]) + ' ' + str(years[ii]))

In [89]:
labels2[1]

['11658.0 2000',
 '15930.0 2001',
 '11308.0 2002',
 '4582.0 2003',
 '2357.0 2004',
 '2504.0 2005',
 '3557.0 2006',
 '3699.0 2007',
 '6810.0 2008',
 '8756.0 2009',
 '9931.0 2010',
 '11992.0 2011',
 '11315.0 2012',
 '12338.0 2013',
 '15315.0 2014',
 '41486.0 2015',
 '139350.0 2016',
 '25011.0 2017',
 '0.0 2018']

In [91]:
import plotly.graph_objs as go
sent_year = []
# colors = ['rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)']
# colors = 'inferno'

labels2 = []
for i in range(len(sent_each_year)):
    labels2.append([])
    for ii in range(len(origins)):
        labels2[i].append(str(sent_each_year[i][ii]) + ' ' + origins[ii])

cmap = plt.get_cmap('BuPu')
# colors = [cmap(np.linspace(0, i/len(origins), len(years))) for i in range(len(years))]
# colors = cmap(np.linspace(0, 1, 18))

for i in range(len(origins)):
    sent_year.append(go.Scattergeo(
        lon = org_lons,
        lat = org_lats,
        text = labels2[i],
        name = str(years),
        marker = dict(
            size = relative_sizes[i],
            color = colors[i]
#             line = dict(width = 0)
        ),
    ) )

# cases[0]['text'] = df[ df['Month'] == 9 ]['Value'].map('{:.0f}'.format).astype(str)+' '+\
#     df[ df['Month'] == 9 ]['Country']
# cases[0]['mode'] = 'markers+text'
# cases[0]['textposition'] = 'bottom center'

inset = [
    go.Choropleth(
#         locationmode = 'country codes',
        locations = org_codes,
        z = sent_each_year,
#         text = df[ df['Month'] == 9 ]['Country'],
        colorscale = [[0,'rgb(0, 0, 0)'],[1,'rgb(0, 0, 0)']],
        autocolorscale = False,
        showscale = False,
        geo = 'geo'
    ),
    go.Scattergeo(
        lon = [21.0936],
        lat = [7.1881],
        text = ['World'],
        mode = 'text',
        showlegend = False,
        geo = 'geo'
    )
]

layout = go.Layout(
    title = 'Regufees each year',
    geo = dict(
        resolution = 50,
        scope = 'asia',
        showframe = False,
        showcoastlines = True,
        showland = True,
        showcountries=True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'mercator'
        )#,
#         lonaxis = dict( range= [ -15.0, -5.0 ] ),
#         lataxis = dict( range= [ 0.0, 12.0 ] ),
#         domain = dict(
#             x = [ 0, 1 ],
#             y = [ 0, 1 ]
#         )
    )#,
#     geo2 = dict(
#         scope = 'world',
#         showframe = False,
#         showland = True,
#         landcolor = "rgb(229, 229, 229)",
#         showcountries = False,
#         domain = dict(
#             x = [ 0, 0.6 ],
#             y = [ 0, 0.6 ]
#         ),
#         bgcolor = 'rgba(255, 255, 255, 0.0)',
#     ),
#     legend = dict(
#            traceorder = 'reversed'
#     )
)

fig = go.Figure(layout=layout, data=sent_year)
ply.iplot(fig, validate=False, filename='Refugees_sent_each_year2')

In [99]:
colors = [cmap(i/len(years)) for i in range(len(years))]

In [100]:
colors

[(0.9686274509803922, 0.9882352941176471, 0.9921568627450981, 1.0),
 (0.931841599384852, 0.962645136485967, 0.977762399077278, 1.0),
 (0.8950557477893117, 0.9370549788542869, 0.963367935409458, 1.0),
 (0.845444059976932, 0.9004998077662437, 0.9428681276432141, 1.0),
 (0.7926643598615917, 0.8605151864667435, 0.920476739715494, 1.0),
 (0.7358246828143022, 0.8182545174932718, 0.8971626297577855, 1.0),
 (0.683044982698962, 0.7814686658977317, 0.8779700115340253, 1.0),
 (0.6262053056516724, 0.7418531334102269, 0.8573010380622837, 1.0),
 (0.5944175317185698, 0.6840753556324491, 0.8269127258746636, 1.0),
 (0.5634140715109573, 0.6186236063052672, 0.7924644367550941, 1.0),
 (0.5490196078431373, 0.5538485198000769, 0.759677047289504, 1.0),
 (0.5490196078431373, 0.47978469819300285, 0.7235063437139563, 1.0),
 (0.5482199154171473, 0.41121107266435986, 0.6901191849288735, 1.0),
 (0.5413302575932334, 0.33886966551326414, 0.6556708958093042, 1.0),
 (0.5349327181853133, 0.2716955017301038, 0.623683198

In [53]:
relative_sizes2

[[23.563394683026583,
  4.182992465016146,
  0.43061179442086334,
  38.0374479889043,
  7.020965382740127],
 [38.16462167689161,
  5.715823466092573,
  0.46516929556976455,
  50.0,
  9.2442710872745],
 [50.0,
  4.057409400789379,
  0.36728653053544225,
  23.54368932038835,
  8.839590443686006],
 [41.48261758691206,
  1.6440617151058485,
  0.29002970859522853,
  23.613037447988905,
  9.92686494392979],
 [33.287321063394685,
  0.8457122353785432,
  0.24950153970855904,
  17.40638002773925,
  11.647976596782058],
 [25.833333333333332,
  0.8984571223537854,
  0.25981451125561333,
  14.147018030513177,
  8.615309605070697],
 [18.834355828220858,
  1.2762827412988877,
  0.16953077788754156,
  19.105409153952845,
  7.4646513895660656],
 [16.232106339468302,
  1.3272335844994618,
  0.17369215237144067,
  24.02912621359223,
  5.494880546075085],
 [18.491820040899796,
  2.4434876210979546,
  0.20390734971105587,
  23.231622746185852,
  7.430521696733301],
 [18.721881390593047,
  3.14172945819878

In [135]:
colors = ['blue',
            'blueviolet', 'burlywood',
            'chartreuse', 'chocolate', 'coral', 
            'crimson',
            'darkgoldenrod', 'darkgreen',
            'darkmagenta', 'darkolivegreen', 'darkorange',
            'darkred', 
            'deeppink',
            'forestgreen', 'fuchsia', 'gainsboro',
            'gray','green']

In [136]:
colors = cmap(np.linspace(0, 1, 18))

In [134]:
len(colors)

18

In [150]:
colors = ['red','crimson', 'deeppink', 'fuchsia', 'violet', \
         'darkviolet', 'slateblue', 'darkblue','slategrey', \
          'skyblue', 'darkslategrey',
         'mediumseagreen', 'green','lawngreen', 'orange', 'peachpuff',\
         'coral', 'lightcoral']

In [143]:
len(colors)

18